---
title: "Perturbation of Neoclassical Model"
author: "Pablo Winant"
format:
    html: default
    ipynb: default
---

In [ ]:
Our goal here is to compute a linear approximation of solution to the neoclassical model, close ot the steady-state.

__Warm-up: install the `ForwardDiff` library. Use it to differentiate the function below. Check the jacobian function.__

Note: the signature of function `f` needs to be fixed first to allow for dual numbers as arguments.

In [16]:
# function f(x::Vector{T}) where T <: Number
function f(x::Vector{<:Number}) # equivalent
    a = x[1]
    b = x[2]
    x1 = a+b
    x2 = a*exp(b)
    return [x1,x2]
end

f (generic function with 4 methods)

In [17]:
using ForwardDiff

In [18]:
ForwardDiff.jacobian(f, [0.2, 0.4])

2×2 Matrix{Float64}:
 1.0      1.0
 1.49182  0.298365

__Create a NamedTuple to hold the model parameters.__

In [1]:
parameters = (;
    α=0.3,
    β=0.96,
    γ=4.0,
    δ=0.1,
    ρ=0.9
)

(α = 0.3, β = 0.96, γ = 4.0, δ = 0.1, ρ = 0.9)

__Define two functions:__
- `transition(z::Number, k::Number, i::Number, p)::Tuple{Number}` which returns productivity and capital at date `t+1` as a function of productivity, capital and investment at date `t`
- `arbitrage(z::Number, k::Number, i::Number, Z::Number, K::Number, I::Number, p)::Number` which returns the residual of the euler equation (lower case variable for date t, upper case for date t+1)


In [2]:
function transition(z::Number, k::Number, i::Number, p)

        Z = p.ρ * z
        K = (1-p.δ) * k + i

        (;Z, K)

end

transition (generic function with 1 method)

In [3]:
function arbitrage(z::Number, k::Number, i::Number, Z::Number, K::Number, I::Number, p)

    # positional unpacking (error-prone)
    # α, β, γ, δ, ρ = p
    
    (;α, β, γ, δ, ρ) = p

    # define auxiliary variables today
    y = exp(z)k^α
    c = y - i

    # define auxiliary variables tomorrow
    Y = exp(Z)K^α
    C = Y - I

    residual = β*(C/c)^(-γ)*( (1-δ) + α*K^(α-1)*exp(Z)) - 1

    return residual


end

arbitrage (generic function with 1 method)

__Using multiple dispatch, define two variants of the same functions, that take vectors as input and output arguments:__
- `arbitrage(s::Vector{T}, x::Vector{T}, S::Vector{T}, X::Vector{T}, p) where T<:Number`
- `transition(s::Vector{T}, x::Vector{T}, p) where T<:Number`

In [4]:
# this returns a number
# arbitrage(s::Vector{T}, x::Vector{T}, S::Vector{T}, X::Vector{T}, p) where T<:Number = arbitrage(s[1],s[2],x[1],S[1],S[2],X[1],p)

In [5]:
[2.4]   # create a vector from  a number

1-element Vector{Float64}:
 2.4

In [6]:
arbitrage(s::Vector{T}, x::Vector{T}, S::Vector{T}, X::Vector{T}, p) where T<:Number = [
    arbitrage(s[1],s[2],x[1],S[1],S[2],X[1],p)
]

arbitrage (generic function with 2 methods)

In [7]:
# this returns a tuple
# transition(s::Vector{T}, x::Vector{T}, p) where T<: Number = transition(s[1], s[2], x[1], p)

In [8]:
t = (1,2,3)

(1, 2, 3)

In [9]:
# to convert into a tuple into a vector
[t...]

3-element Vector{Int64}:
 1
 2
 3

In [10]:
transition(s::Vector{T}, x::Vector{T}, p) where T<: Number = [transition(s[1], s[2], x[1], p)...]

transition (generic function with 2 methods)

__Write a function `steady_state(p)::Tuple{Vector,Vector}` which computes the steady-state of the model computed by hand.__
It returns two vectors, one for the states, one for the controls. Check that the steady-state satisfies the model equations.


In [11]:
function steady_state(p)
    (;α, β, γ, δ, ρ) = p

    # ...
    z = 0.0

    k = ((1/β - (1-δ))/α)^ (1/(α-1))
    i = δ*k

    s = [z,k] # vector of states
    x = [i]  # vector controls

    return (;
        s,
        x
    )


end

steady_state (generic function with 1 method)

In [12]:
q = steady_state(parameters)

(s = [0.0, 2.920822149964071], x = [0.29208221499640713])

In [13]:
# check the steady-state is correct using the functions representing the model


In [14]:
methods(transition)

# 2 methods for generic function "transition" from Main:
 [1] transition(z::Number, k::Number, i::Number, p)
     @ ~/Teaching/ensae/mie37/tutorials/3_perturbation_neoclassical.ipynb:1
 [2] transition(s::Vector{T}, x::Vector{T}, p) where T<:Number
     @ ~/Teaching/ensae/mie37/tutorials/3_perturbation_neoclassical.ipynb:1

In [15]:
@assert maximum(transition(q.s, q.x, parameters) - q.s) == 0.0

In [16]:
@assert maximum(arbitrage(q.s, q.x, q.s, q.x, parameters)) == 0.0

The first order system satisfies:
$$\begin{align}A s_t + B x_t + C s_{t+1} + D x_{t+1} & = & 0 \\\\ 
s_{t+1} & = & E s_t + F x_t
 \end{align}$$

__Define a structure `PerturbedModel` to hold matrices A,B,C,D,E,F.__



In [17]:
struct PerturbedModel
    A::Matrix
    B::Matrix
    C::Matrix
    D::Matrix
    E::Matrix
    F::Matrix

end

__Write a function `first_order_model(s::Vector, x::Vector, p)::PerturbedModel`, which returns the first order model, given the steady-state and the calibration. Suggestion: use `ForwardDiff.jl` library.__

In [18]:
# we need to loosen the constraint on the arbitrage arguments:

# brutal
arbitrage(s, x, S, X, p) where T<:Number = [
    arbitrage(s[1],s[2],x[1],S[1],S[2],X[1],p)
]


# more precise
# arbitrage(s::Vector{<:Number}, x::Vector{<:Number}, S::Vector{<:Number}, X::Vector{<:Number}, p) = [
#     arbitrage(s[1],s[2],x[1],S[1],S[2],X[1],p)
# ]

arbitrage (generic function with 3 methods)

In [19]:
arbitrage(s,x,s,x,parameters)

UndefVarError: UndefVarError: `s` not defined

In [24]:
transition(s, x, p) where T<: Number = [transition(s[1], s[2], x[1], p)...]

transition (generic function with 3 methods)

In [25]:
using ForwardDiff

In [26]:
function first_order_model(s, x, parameters)

    A = ForwardDiff.jacobian(  u->arbitrage(u, x, s, x, parameters), s    )
    B = ForwardDiff.jacobian(  u->arbitrage(s, u, s, x, parameters), x    )
    C = ForwardDiff.jacobian(  u->arbitrage(s, x, u, x, parameters), s    )
    D = ForwardDiff.jacobian(  u->arbitrage(s, x, s, u, parameters), x    )
    E = ForwardDiff.jacobian(  u->transition(u, x, parameters), s    )
    F = ForwardDiff.jacobian(  u->transition(s, u, parameters), x    )

    return PerturbedModel(A,B,C,D,E,F)

end

first_order_model (generic function with 1 method)

In [27]:
@time model = first_order_model(q.s, q.x, parameters)

  1.343581 seconds (7.27 M allocations: 474.892 MiB, 7.63% gc time, 99.95% compilation time)


PerturbedModel([5.074626865671642 0.5212190203776081], [-3.679193085018409;;], [-4.938626865671642 -0.5538125831185546], [3.679193085018409;;], [0.9 0.0; 0.0 0.9], [0.0; 1.0;;])

__We look for a linear solution $x_t = X s_t$ . Write the matrix equation which `X` must satisfy. Write a function `residual(X::Array, M::PerturbedModel)` which computes the residual of this equation for a given `X`.__


In [28]:
function residual(X::Matrix, M::PerturbedModel)
    (;A,B,C,D,E,F) = M # keyword unpacking
    return A + B*X + (C+D*X)*(E+F*X)
end

residual (generic function with 1 method)

In [29]:
X0 = zeros(1, 2)
residual(X0, model)

1×2 Matrix{Float64}:
 0.629863  0.0227877

__Write a function `T(X, M::PerturbedModel)`  which implements the time iteration step.__

In [30]:
function T(X::Matrix, M::PerturbedModel)
    (;A,B,C,D,E,F) = M # keyword unpacking

    C_DX = (C+D*X)

    return -(B+C_DX*F) \ (A+C_DX*E)
    # return  -solve(B+C_DX*F ,   (A+C_DX*E))
    # return  -inv(B+C_DX*F)*(A+C_DX*E)) # not ok
end

T (generic function with 1 method)

In [31]:
T(X0, model)

1×2 Matrix{Float64}:
 0.148798  0.00538334

__Write function `linear_time_iteration(X_0::Matrix, m::PerturbedModel)::Matrix` which implements the time iteration algorithm. Apply it to `X0 = rand(1,2)` and check that the result satisfies the first order model.__

    

In [32]:
A = rand(10000, 10000);
B = rand(10000, 10000);

In [33]:
distance(A,B) = sum(abs(e1-e2) for (e1, e2) in zip(A,B))

distance (generic function with 1 method)

In [34]:
function linear_time_iteration(X_0, M; N=5, τ_η=1e-8, verbose=true)

    η_0 = 1.0

    for n in 1:N

        X = T(X_0, M)

        # successive approximation error
        η = distance(X, X_0)


        # ratio of successive approximation errors
        λ = η/η_0

        if verbose
            println(n, " : ", η, " : ", λ)
        end

        # η_0 will be the value from last iteration
        η_0 = η

        if η<τ_η
            return X
        end

        X_0 = X
    end

    error("No convergence")

end

linear_time_iteration (generic function with 1 method)

In [35]:
@time sol = linear_time_iteration(X0, model; N=500)

1 : 0.15418131543048902 : 0.15418131543048902
2 : 0.12190031209691013 : 0.7906296022741327
3 : 0.0971923112531961 : 0.7973097819136732
4 : 0.07801352484927856 : 0.802671773552593
5 : 0.06295682833902616 : 0.8069988948795506
6 : 0.05102689044058986 : 0.8105060529067174
7 : 0.04150316170984929 : 0.8133586301554283
8 : 0.033853524549360574 : 0.815685435871905
9 : 0.027678228788813052 : 0.8175878038476156
10 : 0.022672513753932016 : 0.8191461211960124
11 : 0.018601088097577975 : 0.8204246030880477
12 : 0.01528032593947912 : 0.8214748438006029
13 : 0.01256560026933672 : 0.8223384971698489
14 : 0.010342108838667052 : 0.8230493264937326
15 : 0.008518120644127804 : 0.8236347902547955
16 : 0.007019930424960394 : 0.8241172810577381
17 : 0.005788038650448812 : 0.8245151020113518
18 : 0.004774224564889556 : 0.8248432419364298
19 : 0.0039392795027168095 : 0.8251139947808338
20 : 0.003251234928149705 : 0.8253374572450166
21 : 0.0026839657341018452 : 0.8255219304097795
22 : 0.0022160813815156506 : 0.

1×2 Matrix{Float64}:
 0.768674  0.0278097

In [36]:
residual(sol, model)

1×2 Matrix{Float64}:
 3.01193e-8  1.08968e-9

__Check blanchard Kahn Conditions__

In [37]:
# check that solution is not diverging
# let's check the transition matrix has spectral radius smaller than 1

In [38]:
using LinearAlgebra

In [48]:
function bk_check(X, M)
    (;A,B,C,D,E,F) = M # keyword unpacking

    P = E+F*X
    maximum(abs,eigvals(P))

    # can check the solution is unique by computing the first "rejected" eigenvalue


end

bk_check (generic function with 1 method)

In [49]:
bk_check(sol, model)


4.13068831324392

__Define two linear operators `L_S(U::Union{Vector, Matrix}, X_0::Matrix, m::PerturbedModel)::Matrix` and `L_T(U::Matrix, X_0::Matrix, m::PerturbedModel)::Matrix` which implement the derivatives of the simulation and the time-iteration operator respectively.__



__Implement a function `spectral_radius(f::Function)::Float64` which implements the power iteration method to compute the biggest eigenvalues of the two previously defined operators. Check that Blanchard-Kahn conditions are met.__

__Write a function `simulate(s0::Vector, X::Matrix, p, T::Int64)::Tuple{Matrix, Matrix}` to simulate the model over $T$ periods (by using the formula $\Delta s_t = (E + F X) \Delta s_{t-1}$. Return a matrix for the states (one line per date) and another matrix for the controls. Bonus: add a keyword option to compute variables levels or log-deviations. If possible, return a DataFrame object.__

In [50]:
P = model.E + model.F*sol

2×2 Matrix{Float64}:
 0.9       0.0
 0.768674  0.92781

__Make some nice plots.__